In [8]:
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
mus_ori = cv2.imread('gambar/mustache.png', -1)

cap = cv2.VideoCapture(0) 
cap.set(cv2.CAP_PROP_FPS, 30)


def transparentOverlay(src, overlay, pos = (0, 0), scale = 1):
    overlay = cv2.resize(overlay, (0, 0), fx = scale, fy = scale)
    h, w, _ = overlay.shape
    rows, cols, _ = src.shape
    y, x = pos[0], pos[1]

    for i in range(h):
        for j in range(w):
            if x + i >= rows or y + j >= cols:
                continue
            alpha = float(overlay[i][j][3] / 255.0)
            src[x + i][y + j] = alpha * overlay[i][j][:3] + (1 - alpha) * src[x + i][y + j]
    return src

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img, 1.2, 5, 0, (120, 120), (350, 350))
    
    for (x, y, w, h) in faces:
        if h > 0 and w > 0:
            mus_symin = int(y + 3.5 * h / 6)
            mus_symax = int(y + 5 * h / 6)
            sh_mus = mus_symax - mus_symin

            face_mus_roi_color = img[mus_symin:mus_symax, x:x + w]
            mustache = cv2.resize(mus_ori, (w, sh_mus), interpolation = cv2.INTER_CUBIC)
            transparentOverlay(face_mus_roi_color, mustache)

    cv2.imshow('Kumis', img)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()